读写文件

加载和存储权重向量和整个模型

对于单个张量，我们可以直接调用load和save函数分别读写它们。 这两个函数都要求我们提供一个名称，save要求将要保存的变量作为输入

通过 torch.save() 函数保存的文件是 PyTorch 专有的二进制文件格式，称为 pickle 格式 或 .pt/.pth 文件。
它们不是可读的文本文件，而是二进制格式
无法使用文本编辑器查看内容

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

用load将文件中的数据读回内存

In [2]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

也可以对张量列表进行操作，然后把它们读回内存

In [3]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

也可以读写从字符串映射到张量的字典

In [4]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

加载和保存模型参数

保存单个权重向量（或其他张量）确实有用，但是如果我们想保存整个模型，并在以后加载它们，单独保存每个向量则会变得很麻烦。 毕竟，我们可能有数百个参数散布在各处。 
因此，深度学习框架提供了内置函数来保存和加载整个网络。 

需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。 
例如，如果我们有一个3层多层感知机，我们需要单独指定架构。因为模型本身可以包含任意代码，所以模型本身难以序列化。 
因此，为了恢复模型，我们需要用代码生成架构，然后从磁盘加载参数，从熟悉的多层感知机为例进行说明

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

编写程序，将模型的参数存放到一个文件中

In [6]:
torch.save(net.state_dict(), 'mlp.params')

如果要恢复模型，实例化一个之前感知机的模型
此时并不需要初始化模型参数，直接读取存储的参数即可

In [ ]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

# eval表示设置成评估模式，使用clone.eval()表示模型保存后载入，加载保存的模型参数后应当设置为评估模式

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [8]:
# 验证是否相同
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])